In [1]:
import datasets
import torch 
import numpy as np
import pandas as pd
import evaluate
from datasets import Dataset
from tqdm import tqdm
from transformers import RobertaTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq



/Users/pippertetsing/miniforge3m1/envs/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
etric = evaluate.load('rouge')

data_path = '../filtered_comment_code_sol.pkl'
df = pd.read_pickle(data_path)[:100]
dataset = Dataset.from_pandas(df)

base_model = "Salesforce/codet5-base"
sol_tok_model = "Pipper/finetuned_sol"
tokenizer = RobertaTokenizer.from_pretrained(base_model)


In [21]:

max_input_length = 256
max_target_length = 128
prefix = "Generate Solidity: "
def process_samples(samples):
    codes = samples['code_string']
    comments = samples['comments']

    inputs = [prefix + cm for cm in comments]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    # encode the summaries
    labels = tokenizer(codes, max_length=max_target_length, padding="max_length", truncation=False).input_ids

    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

dataset = dataset.map(process_samples, batched=True, num_proc=5)

In [22]:

com = dataset[0]['input_ids']
cod = labels = dataset[0]['labels']
print(tokenizer.decode([label for label in labels if label != -100]))

<s>contract DumplingERC721 is ERC721, Ownable, PricingCalculator {


    uint public constant MAX_DUMPLINGS = 2500;
    bool public hasSaleStarted = true;

    string public constant R = "We are nervous. Are you?";

pragma solidity ^0.7.4;
    constructor (string memory name, string memory symbol, string memory baseURI) public ERC721(name, symbol){
        _setBaseURI(baseURI);
    }

    function setBaseURI(string memory baseURI) public onlyOwner {
        _setBaseURI(baseURI);
    }
    function calculatePrice() public view returns (uint256) {
        require(hasSaleStarted == true, "Sale hasn't started");
        require(totalSupply() < MAX_DUMPLINGS, "No more dumplings");

        uint currentSupply = totalSupply();

        uint currentPrice =  priceCalculator(currentSupply);
        return currentPrice;

    }

    function calculatePriceForToken(uint _id) public view returns (uint256) {
        require(_id < MAX_DUMPLINGS, "Sale has already ended");

        uint currentPrice = 

In [9]:
cod = labels = dataset[0]['labels']
